In [4]:
import numpy as np
import numpy.random as rnd
import scipy
import pandas as pd
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.optimize import minimize
from numpy import linalg as LA
import cvxpy as cp
import matplotlib.pyplot as plt

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [5]:
from exp_setup import Experiment

In [6]:
from algorithms import *

In [7]:
num_species = 4 #drone,rover,mini-rover,mini-drone
num_tasks = 3  #pick,search for target,move object
num_traits = 9 #speed,footprint,payload,reach,weight,sensing frequency,sensing range,color,battery capacity
traits = ["speed","footprint","payload","reach","weight","sensing frequency","sensing range","color","battery"]
num_demo = 1000

In [8]:
exp = Experiment()
D = exp.get_expert_demonstrations()

In [9]:
new_Q = exp.get_random_q()
agents_target = exp.get_agent_limit()
Q = D["Q"]
Y = D["Y"]
Y_mean = np.mean(Y, axis=0)
new_Q

array([[ 10.05553949,  93.84137514,   5.25742525,   1.15256053,
         24.2699095 , 197.21472038,  57.43590637,   2.        ,
         12.        ],
       [  3.9695241 , 134.68015214,   5.46816942,   4.05992183,
         25.44392767, 252.28090306,  57.37324835,   3.        ,
         17.        ],
       [  7.87356778, 119.13650485,  15.59548752,   3.36700834,
         24.51701411, 208.86945659,  58.29748928,   0.        ,
         13.        ],
       [ 14.76779327,  74.35324866,  11.35955227,   1.38968931,
         22.79962149, 211.2746862 ,  57.33840902,   3.        ,
         16.        ]])

In [10]:
X_mat = solve_task_allocation(agents_target,new_Q,Y_mean)
print("mat" , X_mat)

mat [[ 2.  0.  0. 10.]
 [ 8.  1.  5.  0.]
 [ 0.  1.  3.  0.]]


In [11]:
X_per_task = solve_task_allocation_per_task(agents_target,new_Q,Y_mean)
print("task based" , X_per_task)

task based [array([ 2.,  0.,  0., 10.]), array([10.,  2.,  2., -0.]), array([0., 1., 3., 0.])]


In [12]:
Y_mat = X_mat@new_Q
print(Y_mat)

Y_per_task = X_per_task@new_Q
print(Y_per_task)

[[ 167.78901168  931.21523685  124.11037316   16.20201412  276.53603388
  2507.17630276  688.25590292   34.          184.        ]
 [ 123.78167893 1481.09367747  125.505009     30.1154478   342.18827422
  2874.34594909  808.34794574   19.          178.        ]
 [  27.59022743  492.08966668   52.25463198   14.16094686   98.99496999
   878.88927284  232.26571619    3.           56.        ]]
[[ 167.78901168  931.21523685  124.11037316   16.20201412  276.53603388
  2507.17630276  688.25590292   34.          184.        ]
 [ 124.24157869 1446.04706535   94.70156635   26.37946567  342.62097858
  2894.44792314  805.700539     26.          180.        ]
 [  27.59022743  492.08966668   52.25463198   14.16094686   98.99496999
   878.88927284  232.26571619    3.           56.        ]]


In [13]:
# optimal_Y = np.array([[ 122.53839519, 0., 119.39044525, 32.15983648, 360.87624558, 2920.25543052, 768.93055775, 0., 0.],
#                                    [ 120.64147124, 1433.4094241 ,  0.,   0., 358.13547136, 2906.93229221,  766.11629311,   0., 190.],
#                                    [ 122.31492305, 1441.74082593,  119.06910271,   0., 358.78852644, 0. ,  769.11520065,   0., 193.]])
optimal_Y = exp.get_optimal_Y()

In [14]:
LA.norm(optimal_Y-Y_per_task, 2)/LA.norm(optimal_Y,2)

0.3800926205942008

In [15]:
LA.norm(optimal_Y-Y_mat, 2)/LA.norm(optimal_Y,2)

0.38033700192188513

Baseline : Weighted optimization

In [29]:
Q_D = np.concatenate(Q)
var_N = np.var(Q_D,axis=0)
var_O = np.var(Y,axis=0)
print(var_O)
print(var_N)

[[ 542.19574868 7352.11974886  454.78414421  404.22988833  660.02095334
  6011.06031257 1645.93733022  196.0255839   877.0969751 ]
 [ 224.30108478 1156.26341011  766.19473029   97.64859299  649.71067912
  3197.9249996  1220.76916463  166.062636    778.54375658]
 [  25.28578288 4314.70272239  172.62686286   17.01016715  148.63996863
  3219.62047118  466.85288685   30.82281603  150.84150377]]
[1.61284264e+01 1.13233642e+03 2.00037653e+01 2.02320772e+00
 3.01499114e+00 2.77705823e+02 9.06265597e-01 1.98499100e+00
 8.13375900e+00]


In [17]:
def calculate_weight(natural_variance, observerd_variance):
    weights = []
    for i in range(num_traits):
        weights.append((natural_variance[i]/2)*math.cos(2*observerd_variance[i] + 0.5) + 0.5)
    
    return weights

In [18]:
cv_N = np.sqrt(var_N)/np.mean(Q_D,axis=0)
cv_O = np.sqrt(var_O)/Y_mean

norm_var_N = cv_N/np.sum(cv_N)
norm_var_O = cv_O
for m in range(num_tasks):
    norm_var_O[m] /= np.sum(cv_O[m])

print(norm_var_O)
# print(np.sum(norm_var_O,axis=1))
# print(np.sum(norm_var_N))

[[0.06882699 0.05156232 0.12437458 0.29508669 0.0399809  0.01433228
  0.02905381 0.29182208 0.08496035]
 [0.0902492  0.01561608 0.16798976 0.21326497 0.04796768 0.01310454
  0.03021464 0.32468382 0.09690931]
 [0.10069277 0.04467602 0.2255371  0.14520248 0.04670924 0.02796126
  0.03808487 0.28481125 0.086325  ]]


In [53]:
w = [calculate_weight(norm_var_N,norm_var_O[m]) for m in range(num_tasks)]
print(w)
sum_w = np.sum(w,axis=1)

for m in range(num_tasks):
    w[m] /= sum_w[m]

# print(w)

[[0.5609075392151393, 0.5451018461240199, 0.5634794435508346, 0.5455148070911042, 0.5094227410500946, 0.5114381552999363, 0.5024116142548998, 0.5574926767885251, 0.5267748834925668], [0.5589188130062414, 0.5472166258099755, 0.5580989695911684, 0.5591279923286756, 0.50932293890205, 0.5114545266080152, 0.5024081122301616, 0.5502406561587322, 0.5262604341292952], [0.5579098432708935, 0.5455253755291554, 0.5503308826536512, 0.5692625947519756, 0.5093388227118864, 0.5112518100707041, 0.5023840278210199, 0.5590087582696263, 0.5267168979311081]]
[array([0.11630948, 0.11303202, 0.11684279, 0.11311765, 0.10563362,
       0.10605153, 0.10417979, 0.11560137, 0.10923175]), array([0.11588495, 0.11345865, 0.11571497, 0.11592832, 0.10560186,
       0.10604382, 0.10416815, 0.11408564, 0.10911364]), array([0.11546795, 0.1129048 , 0.11389937, 0.11781757, 0.10541544,
       0.10581136, 0.10397604, 0.11569539, 0.10901209])]


In [20]:
X = []
for m in range(num_tasks):
        X_sol = cp.Variable((num_species), integer=True)

        # minimize trait mismatch
        mismatch_mat = w[m]*Y_mean[m] - cp.multiply(w[m],cp.matmul(new_Q.T,X_sol))  # trait mismatch matrix
        # print(mismatch_mat.shape)
        # weighted_mat = cp.matmul(mismatch_mat, cp.matmul(W[m],mismatch_mat.T))
        # print(w[m],mismatch_mat)
        obj = cp.Minimize(cp.norm2(mismatch_mat))
        # ensure each agent is only assigned to one task
        constraints = [X_sol <= np.array(agents_target), X_sol >= 0]

        # solve for X_target
        opt_prob = cp.Problem(obj, constraints)
        opt_prob.solve(solver=cp.CPLEX)
        X_target = X_sol.value
        X.append(np.round(X_target))
X

[array([ 2.,  0.,  0., 10.]),
 array([10.,  2.,  2., -0.]),
 array([0., 1., 3., 0.])]

In [21]:
Y_baseline1 = X@new_Q
LA.norm(optimal_Y-Y_baseline1, 2)/LA.norm(optimal_Y,2)

0.3800926205942008

Baseline : Weighted optimization - Reduced Dimensions

In [50]:
w_new = w
for m in range(num_tasks):
    idx = np.argpartition(w[m], 3)
    print(idx)
    for i in range(3):
        w_new[m][idx[i]] = 0
sum_w_new = np.sum(w_new,axis=1)

for m in range(num_tasks):
    w_new[m] /= sum_w_new[m]

print(w_new)

[5 4 6 8 1 3 2 7 0]
[5 4 6 8 1 3 2 7 0]
[5 4 6 8 1 3 2 7 0]
[array([0.20231137, 0.19661049, 0.20323902, 0.19675943, 0.        ,
       0.        , 0.        , 0.20107968, 0.        ]), array([0.20151363, 0.1972945 , 0.20121804, 0.20158905, 0.        ,
       0.        , 0.        , 0.19838479, 0.        ]), array([0.20054002, 0.19608844, 0.19781577, 0.20462075, 0.        ,
       0.        , 0.        , 0.20093502, 0.        ])]


In [51]:
X_baseline2 = []
for m in range(num_tasks):
        X_sol = cp.Variable((num_species), integer=True)
        # minimize trait mismatch
        mismatch_mat = w_new[m]*Y_mean[m] - cp.multiply(w_new[m],cp.matmul(new_Q.T,X_sol))  # trait mismatch matrix
        # print(mismatch_mat)
        # weighted_mat = cp.matmul(mismatch_mat, cp.matmul(W[m],mismatch_mat.T))
        # print(w[m],mismatch_mat)
        obj = cp.Minimize(cp.norm2(mismatch_mat))
        # ensure each agent is only assigned to one task
        constraints = [X_sol <= np.array(agents_target), X_sol >= 0]

        # solve for X_target
        opt_prob = cp.Problem(obj, constraints)
        opt_prob.solve(solver=cp.CPLEX)
        X_target = X_sol.value
        X_baseline2.append(np.round(X_target))
X_baseline2

[array([3., 0., 0., 7.]), array([3., 5., 3., 2.]), array([2., 3., 0., 0.])]

In [52]:
Y_baseline2 = X_baseline2@new_Q
LA.norm(optimal_Y-Y_baseline2, 2)/LA.norm(optimal_Y,2)

0.4202730929566889